<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/Stocks_pama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
pip install requests beautifulsoup4 pandas


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pytz
from IPython.display import display, clear_output, HTML  # Jupyter Notebook 출력 관리

# 🔹 한국 시간대 설정
kst = pytz.timezone("Asia/Seoul")

# 🔹 네이버 금융에서 실시간 주가 가져오기
def get_naver_stock_price(stock_code):
    url = f"https://finance.naver.com/item/main.naver?code={stock_code}"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        price_tag = soup.select_one("p.no_today span.blind")
        if price_tag:
            return int(price_tag.text.replace(",", ""))
        else:
            return None
    except requests.RequestException:
        return None

# 🔹 보유 주식 정보
portfolio = [
    {"종목명": "코스모신소재", "종목코드": "005070", "매입가": 164598, "수량": 318},
    {"종목명": "POSCO홀딩스", "종목코드": "005490", "매입가": 522538, "수량": 130},
    {"종목명": "한화오션", "종목코드": "042660", "매입가": 44000, "수량": 300},
    {"종목명": "에코프로비엠", "종목코드": "247540", "매입가": 499538, "수량": 70}
]

# 🔹 실시간 모니터링 (Jupyter Notebook 최적화)
try:
    while True:
        # 이전 출력 삭제 (Jupyter Notebook)
        clear_output(wait=True)

        # 현재 한국시간
        current_time = datetime.now(kst).strftime("[%Y-%m-%d %H:%M:%S]")

        # HTML 테이블 시작
        html_output = f"<h3>{current_time}</h3>"
        html_output += """
        <table border="1" style="border-collapse: collapse; width: 100%; text-align: center;">
            <tr style="background-color: #f2f2f2;">
                <th>종목명</th>
                <th>매입가</th>
                <th>현재가</th>
                <th>수량</th>
                <th>손익(원)</th>
                <th>수익률(%)</th>
            </tr>
        """

        for stock in portfolio:
            current_price = get_naver_stock_price(stock["종목코드"])
            if current_price is None:
                continue

            purchase_price = stock["매입가"]
            quantity = stock["수량"]

            total_purchase = purchase_price * quantity
            total_current = current_price * quantity
            profit_loss = total_current - total_purchase
            profit_percent = (profit_loss / total_purchase) * 100

            # 🔹 손익 및 수익률에 색상 적용 (빨간색: 플러스, 파란색: 마이너스)
            profit_color = "red" if profit_loss >= 0 else "blue"
            percent_color = "red" if profit_percent >= 0 else "blue"

            profit_loss_html = f'<span style="color: {profit_color};">{profit_loss:+,}</span>'
            profit_percent_html = f'<span style="color: {percent_color};">{profit_percent:+.2f}%</span>'

            # HTML 테이블 행 추가
            html_output += f"""
            <tr>
                <td>{stock['종목명']}</td>
                <td>{purchase_price:,}</td>
                <td>{current_price:,}</td>
                <td>{quantity:,}</td>
                <td>{profit_loss_html}</td>
                <td>{profit_percent_html}</td>
            </tr>
            """

        # HTML 테이블 종료
        html_output += "</table>"

        # 최신 결과 출력 (HTML 방식)
        display(HTML(html_output))

        time.sleep(2)  # 2초마다 업데이트

except KeyboardInterrupt:
    print("\n📌 프로그램 종료.")


종목명,매입가,현재가,수량,손익(원),수익률(%)
코스모신소재,"164,598","49,600",318,"-36,569,364",-69.87%
POSCO홀딩스,"522,538","248,500",130,"-35,624,940",-52.44%
한화오션,"44,000","77,600",300,"+10,080,000",+76.36%
에코프로비엠,"499,538","128,400",70,"-25,979,660",-74.30%


In [37]:
import sys
import time

for i in range(10):
    sys.stdout.write("\033[H\033[J")  # 화면 전체 삭제
    sys.stdout.flush()
    print(f"최신 데이터: {i}")  # 새로운 결과 출력
    time.sleep(1)  # 1초 대기


최신 데이터: 0
최신 데이터: 1
최신 데이터: 2
최신 데이터: 3
최신 데이터: 4
최신 데이터: 5
최신 데이터: 6
최신 데이터: 7
최신 데이터: 8
최신 데이터: 9


In [7]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Google Sheets API 인증 설정
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("your-credentials-file.json", scope)
client = gspread.authorize(creds)

# 스프레드시트 열기 (파일명 또는 ID 입력)
spreadsheet = client.open("RealTime_Stock")  # Google Sheets 파일명
sheet = spreadsheet.sheet1  # 첫 번째 시트 선택

# 데이터 업데이트 (A1 셀에 "Hello, Colab!" 입력)
sheet.update("A1", "Hello, Colab!")

print("Google Sheets 업데이트 완료!")


FileNotFoundError: [Errno 2] No such file or directory: 'your-credentials-file.json'